In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np


CIFAR_DIR = ".\deep_learn\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))



['.ipynb_checkpoints', 'batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)


(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool, [])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
data_aug_1 = tf.image.random_flip_left_right(x_image)
data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=63)
data_aug_3 = tf.image.random_contrast(data_aug_2, lower=0.2, upper=1.8)

normal_data_aug_3 = data_aug_3 / 127.5 - 1

def conv_wrapper(inputs, 
                 name, 
                 is_training,
                 output_channel=32, 
                 kernel_size=(3,3),
                 activation=tf.nn.relu,
                 padding = 'same'):
    """wrapper of tf.laysers.conv2d"""
    # without bn: conv  -> activation
    # with batch normalization: conv -> bn -> activation
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(inputs,
                            output_channel,
                            kernel_size,
                            padding= padding,
                            activation= None,
                            name= name+'/conv2d')
        bn = tf.layers.batch_normalization(conv2d,
                                           training= is_training)
        return activation(bn)

def pooling_warpper(inputs, name):
    return tf.layers.max_pooling2d(inputs,
                                   (2,2),
                                   (2,2),
                                   name=name)
# conv1: 神经元图， feature_map, 输出图像

conv1_1 = conv_wrapper(normal_data_aug_3, name = 'conv1_1', is_training=True)
conv1_2 = conv_wrapper(conv1_1, name = 'conv1_2', is_training=True)
conv1_3 = conv_wrapper(conv1_2, name = 'conv1_3', is_training=True)
pooling1 = pooling_warpper(conv1_3, name = 'pool1')


conv2_1 = conv_wrapper(pooling1, name = 'conv2_1', is_training=True)
conv2_2 = conv_wrapper(conv2_1, name = 'conv2_2', is_training=True)
conv2_3 = conv_wrapper(conv2_2, name = 'conv2_3', is_training=True)
pooling2 = pooling_warpper(conv2_3, name = 'pool2')


conv3_1 = conv_wrapper(normal_data_aug_3, name = 'conv3_1', is_training=True)
conv3_2 = conv_wrapper(conv3_1, name = 'conv3_2', is_training=True)
conv3_3 = conv_wrapper(conv3_2, name = 'conv3_3', is_training=True)
pooling3 = pooling_warpper(conv3_3, name = 'pool1')



# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv1_3, 'conv1_3')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv2_3, 'conv2_3')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
    variable_summary(conv3_3, 'conv3_3')

    
    
loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

# source_image = (x_image + 1) * 127.5
inputs_summary = tf.summary.image('inputs_image', normal_data_aug_3)

megred_summary = tf.summary.merge_all()
megred_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = 'run_vgg_tensorboard_data_aug_deeper-bn'

run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')

if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)

if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

model_dir = os.path.join(run_dir, 'model')
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
saver = tf.train.Saver()
model_name = 'ckp-1000'
model_path = os.path.join(model_dir, model_name)

In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 4000
test_steps = 100

output_summary_every_steps = 100
output_model_every_steps = 100

# train 10k: 73.4%
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    
    if os.path.exists(model_path + '.index'):
        saver.restore(sess, model_path)
        print('model restore from {}'.format(model_path))
    else:
        print('model {} does not exist'.format(model_path))
    
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(megred_summary)
        
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels,
                is_training: True
            })
        loss_val, acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summarys_str = sess.run([megred_summary_test],
                                         feed_dict={
                                             x: fixed_test_batch_data,
                                             y: fixed_test_batch_labels,
                                             is_training: False,
                                         })[0]
            test_writer.add_summary(test_summarys_str, i+1)
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            # test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels,
                        is_training:False,
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))
        if (i+1) % output_model_every_steps == 0:
            saver.save(sess, os.path.join(model_dir, 'ckp-{}'.format(i+1)))
            print('model saved to ckp-{}'.format(i+1))
            

model .\run_vgg_tensorboard_data_aug_deeper-bn\model\ckp-1000 does not exist
[Train] Step: 100, loss: 2.09376, acc: 0.40000
model saved to ckp-100
[Train] Step: 200, loss: 2.15312, acc: 0.40000
model saved to ckp-200
[Train] Step: 300, loss: 2.15564, acc: 0.35000
model saved to ckp-300
[Train] Step: 400, loss: 1.58696, acc: 0.45000
model saved to ckp-400
[Train] Step: 500, loss: 1.24729, acc: 0.60000
model saved to ckp-500


KeyboardInterrupt: 